In [1]:
#导包：公共数据包
import csv
import pandas as pd
import time
import json
import numpy as np
from shapely.geometry import LineString
import os
import datetime

#导包：自有包
import utils.util as util
from utils.douglas_peucker import DouglasPeuker
import utils.visualization as visual
from collections import deque

In [2]:
# 需要调整的阈值
LIMIT_LENGTH = 65  # 轨迹长度小于该值的将被过滤掉
LIMIT_POINT = 5  # 轨迹点数小于该值的将被过滤掉

LIMIT_DT = 30  # 时间戳间隔大于该值的将被分段 3 * 平均采样间隔（10s）
LIMIT_DISTANCE = 400  # 距离大于该值的将被分段 2 * 平均采样间隔（10s）* 城市限速（20m/s）
LIMIT_DIRECTION = 35  # 方向变换大于该值的将去噪
DP_THRESHOLD = 5  # 道格拉斯-普克 阈值

In [3]:
# 调用道格拉斯-普克算法，减少轨迹点索引，轨迹抽稀
def get_reduction_traj_index(coordinates):
    traj_mercator = util.transform_points_wgs84_to_mercator(coordinates)
    result_index = []
    d = DouglasPeuker(DP_THRESHOLD)
    d.reduction(traj_mercator)
    temp_list = d.qualify_list
    for i, item in enumerate(traj_mercator):
        if item in temp_list:
            result_index.append(i)
    return result_index  # 抽稀后的坐标点索引

# 计算速度和方向
def calculate_speed_and_dir(traj):
    result = dict()
    for key, value in traj.items():
        directions = util.get_direction(value[1])
        velocities = util.get_velocity(value[1], value[0])
        result[key] = [value[0], value[1], velocities, directions]
    return result

## 1.完成轨迹分段代码实现

In [4]:
def split_traj(traj):  # traj {'id':[[时间戳],[坐标点]]}
    res = {}
    traj_id = 0# 设定一个新的key，用于解决原先的一段轨迹可能被分为多段轨迹的问题
    for key,value in traj.items():
        timestamp = value[0]# 时间戳
        traj_raw = value[1]# 原始坐标点值
        traj_mercator = util.transform_points_wgs84_to_mercator(value[1])# 坐标点转为墨卡托坐标系 list

        if not traj_mercator or not timestamp:
            continue
        time_diff = util.get_time_diff(timestamp)# 时间戳的间隔 list
        distance_diff = util.get_distance_diff(traj_mercator)# 计算相邻坐标点的欧式距离 list
        diff_time_np = np.array(time_diff)
        diff_distance_np = np.array(distance_diff)
        limit_dt_index = np.where(diff_distance_np > LIMIT_DT)
        limit_distance_index = np.where(diff_distance_np > LIMIT_DISTANCE)
        segment_loc_tmp = list(set(limit_dt_index[0]).union(set(limit_distance_index[0])))


        if not segment_loc_tmp and len(traj_mercator) >= LIMIT_POINT:
            # filter 过滤邻近冗余坐标值 返回索引 根据tmp_traj_mercator计算每两个点之间的距离，若其之间的距离小于LIMIT_LENGTH = 65
            # 轨迹长度小于该值的将被过滤掉
            # 过滤方法1
            # filter_distance_index = np.where(diff_distance_np > LIMIT_LENGTH)[0]
            # filter_timestamp = [timestamp[i] for i in filter_distance_index]
            # filter_traj = [traj_raw[i] for i in filter_distance_index]
            # res[str(traj_id)] = [filter_timestamp,filter_traj]
            # traj_id += 1



            # 过滤方法2 从第一个点开始，向后查找点，计算距离，直至点之间的距离大于limit，添加点，循环
            # n = len(traj_mercator)
            # filter_index = []
            # queue = deque(range(1,n))
            #
            # filter_index.append(0)
            #
            # while queue:
            #     a_index = filter_index[-1]
            #     b_index = queue[0]
            #     traj_need_calcul = [traj_mercator[a_index],traj_mercator[b_index]]
            #     distance = util.get_distance_diff(traj_need_calcul)[1]
            #     if distance > LIMIT_LENGTH:
            #         filter_index.append(queue.popleft())
            #     else:
            #         queue.popleft()
            # if len(filter_index) == 1:
            #     filter_index.append(n-1)
            #
            # filtered_timestamp = [timestamp[index] for index in filter_index]
            # filtered_traj = [traj_raw[index] for index in filter_index]
            # res[str(traj_id)] = [filtered_timestamp,filtered_traj]
            # traj_id += 1

            res[str(traj_id)] = [timestamp,traj_raw]# res保存不需要切割的轨迹点
            traj_id += 1
        else:# 如果需要切割
            segment_loc_tmp.sort()
            segment_loc = [0] + segment_loc_tmp + [len(diff_time_np)]
            for index in range(len(segment_loc) - 1):
                if (segment_loc[index + 1]) - segment_loc[index] < LIMIT_POINT:# 过滤短轨迹段，轨迹点数小于5
                    continue
                tmp_traj_mercator = traj_mercator[segment_loc[index]:(segment_loc[index + 1])]# 获取对应轨迹段
                tmp_timestamp = timestamp[segment_loc[index]:(segment_loc[index + 1])]
                tmp_traj_raw = traj_raw[segment_loc[index]:(segment_loc[index + 1])]

                res[str(traj_id)] = [tmp_timestamp,tmp_traj_raw]
                traj_id += 1

                # filter 过滤邻近冗余坐标值 返回索引 根据tmp_traj_mercator计算每两个点之间的距离，若其之间的距离小于LIMIT_LENGTH = 65
                # 轨迹长度小于该值的将被过滤掉

                # tmp_distance_diff = util.get_distance_diff(tmp_traj_mercator)
                # tmp_distance_diff_np = np.array(tmp_distance_diff)
                # limit_tmp_distance_diff = np.where(tmp_distance_diff_np > LIMIT_LENGTH)
                # filtered_timestamp = [tmp_timestamp[i] for i in limit_tmp_distance_diff]
                # filtered_traj_raw = [tmp_traj_raw[i] for i in limit_tmp_distance_diff]
                #
                # res[str(traj_id)] = [filtered_timestamp,filtered_traj_raw]
                # traj_id += 1

    # 待填充
    return res

## 2.完成轨迹去噪代码实现

In [5]:
# 轨迹去噪
def denoise_traj(traj):  # traj {'id':[[时间戳],[坐标点],[方向],[速度]]}
    res = {}
    traj_with_dir_speed = calculate_speed_and_dir(traj)
    for key,value in traj_with_dir_speed.items():
        index = [0]
        direction = value[3]
        for i in range(1,len(value[3]) - 1):
            dir_changed1 = direction[i] - direction[i - 1]
            dir_changed2 = direction[i + 1] + 360.0 - direction[i]
            dir_changed = min(dir_changed1,dir_changed2)
            if dir_changed < LIMIT_DIRECTION:
                index.append(i)
        if len(index) <= 1:
            index.append(len(direction) - 1)

        filter_timestamp = [value[0][i] for i in index]
        filter_traj = [value[1][i] for i in index]
        res[key] = [filter_timestamp,filter_traj]
    # 待填充
    return res

## 3.完成轨迹简化代码实现

In [6]:
## 3.完成轨迹简化代码实现# 轨迹去噪
def simplify_traj(traj):  # traj {'id':[[时间戳],[坐标点],[方向],[速度]]}
    res = {}
    for key,value in traj.items():
        index = get_reduction_traj_index(value[1])
        timestamp = [value[0][i] for i in index]
        traj = [value[1][i] for i in index]
        res[key] = [timestamp,traj]

    # 待填充
    return res

## 4.主函数

In [7]:
traj_dict = json.load(open('traj_dict.json'))
traj_split = split_traj(traj_dict)
traj_denoise = denoise_traj(traj_split)
traj_simplify = simplify_traj(traj_denoise)

In [8]:
count_of_before_spilt = 0
for key,value in traj_dict.items():
    count_of_before_spilt += len(value[1])
print(count_of_before_spilt)

1173410


In [9]:
count_of_after_split = 0
for key,value in traj_split.items():
    count_of_after_split += len(value[1])
print(count_of_after_split)
print(count_of_after_split/count_of_before_spilt)

813111
0.692947051755141


In [10]:
count_of_after_denoise = 0
for key,value in traj_denoise.items():
    count_of_after_denoise += len(value[1])
print(count_of_after_denoise)
print(count_of_after_denoise/count_of_before_spilt)

688616
0.58685029103212


In [11]:
count_of_after_simplify = 0
for key,value in traj_simplify.items():
    count_of_after_simplify += len(value[1])
print(count_of_after_simplify)
print(count_of_after_simplify/count_of_before_spilt)

216613
0.18460129025660255


In [12]:
visual.visual_raw_traj(visual.geo_json_generate_traj_from_dict(traj_dict),"before.json")
visual.visual_raw_traj(visual.geo_json_generate_traj_from_dict(traj_simplify),"after.json")

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
import folium
from folium import PolyLine, Marker

# 取第一个轨迹的第一个点作为初始地图中心
first_traj = next(iter(traj_simplify.values()))
initial_point = first_traj[1][0]  # 第一个坐标点
m = folium.Map(location=initial_point, zoom_start=13)

for traj_id, traj_data in traj_simplify.items():
    coordinates = traj_data[1]  # 获取轨迹坐标
    # 绘制轨迹线条
    PolyLine(
        locations=coordinates,  # 坐标点列表
        color="blue",          # 轨迹颜色
        weight=2,              # 线条宽度
        opacity=0.8            # 透明度
    ).add_to(m)
    # 在轨迹起点添加标记
    Marker(location=coordinates[0], popup=f"Start of {traj_id}", icon=folium.Icon(color='green')).add_to(m)
    # 在轨迹终点添加标记
    Marker(location=coordinates[-1], popup=f"End of {traj_id}", icon=folium.Icon(color='red')).add_to(m)

m.save("traj_map.html")

In [ ]:
# 取第一个轨迹的第一个点作为初始地图中心
first_traj = next(iter(traj_dict.values()))
initial_point = first_traj[1][0]  # 第一个坐标点
m = folium.Map(location=initial_point, zoom_start=13)

for traj_id, traj_data in traj_dict.items():
    coordinates = traj_data[1]  # 获取轨迹坐标
    # 绘制轨迹线条
    PolyLine(
        locations=coordinates,  # 坐标点列表
        color="blue",          # 轨迹颜色
        weight=2,              # 线条宽度
        opacity=0.8            # 透明度
    ).add_to(m)
    # 在轨迹起点添加标记
    Marker(location=coordinates[0], popup=f"Start of {traj_id}", icon=folium.Icon(color='green')).add_to(m)
    # 在轨迹终点添加标记
    Marker(location=coordinates[-1], popup=f"End of {traj_id}", icon=folium.Icon(color='red')).add_to(m)

m.save("traj_map_before.html")

##### 5.轨迹分段超参数实验（提示：距离、时间阈值对分段后剩余轨迹点数量的影响）选做

##### 6.轨迹去噪超参数实验 选做

##### 7.轨迹简化超参数实验 选做